<a href="https://colab.research.google.com/github/JiaxinLi-lipluszn/Organized-Notebook/blob/main/Supervised_bmcite_no_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unzip file

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/scRNA/Stoeckiues/")

# Import

In [3]:
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models
import pandas as pd
from torch.utils.data import Dataset
from functools import partial
import numpy as np
from torch import optim
from pathlib import Path
from torch.autograd import Variable

import os
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
import math
from functools import partial

import torch
import torch.nn as nn


import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

## Set seed

In [4]:
generator=torch.Generator().manual_seed(42)

In [5]:
import utils
import vision_transformer_sc as vits
from utils import trunc_normal_

# Re import files

In [6]:
import os
os.chdir("/content/drive/MyDrive/scRNA/Stoeckiues/")

In [7]:
from importlib import reload  
import vision_transformer_sc as vits  
vits = reload(vits) 

# Read in Expr and meta

In [8]:
expr = pd.read_csv("/content/drive/Shareddrives/Documentation/Data/bm/rna_no_scale.csv", index_col= 0)


In [9]:
meta = pd.read_csv("/content/drive/Shareddrives/Documentation/Data/bm/meta.csv", index_col= 0)


In [10]:
# separate CD4 and CD8 in l1
meta["celltype.l1.5"] = meta["celltype.l1"].values
meta.loc[meta["celltype.l2"].str.startswith("CD4"), "celltype.l1.5"] = "CD4 T"
meta.loc[meta["celltype.l2"].str.startswith("CD8"), "celltype.l1.5"] = "CD8 T"
meta.loc[meta["celltype.l2"]=="Treg", "celltype.l1.5"] = "CD4 T"
meta.loc[meta["celltype.l2"]=="MAIT", "celltype.l1.5"] = "MAIT"
meta.loc[meta["celltype.l2"]=="gdT", "celltype.l1.5"] = "gdT"

In [11]:
meta["perturb"] = meta[meta.columns[-1]]

In [12]:
meta

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,lane,donor,celltype.l1,celltype.l2,RNA.weight,celltype.l1.5,perturb
a_AAACCTGAGCTTATCG-1,bmcite,7546,2136,1350,25,HumanHTO4,batch1,Progenitor cells,Prog_RBC,0.482701,Progenitor cells,Progenitor cells
a_AAACCTGAGGTGGGTT-1,bmcite,1029,437,2970,25,HumanHTO1,batch1,T cell,gdT,0.241789,gdT,gdT
a_AAACCTGAGTACATGA-1,bmcite,1111,429,2474,23,HumanHTO5,batch1,T cell,CD4 Naive,0.507714,CD4 T,CD4 T
a_AAACCTGCAAACCTAC-1,bmcite,2741,851,4799,25,HumanHTO3,batch1,T cell,CD4 Memory,0.431308,CD4 T,CD4 T
a_AAACCTGCAAGGTGTG-1,bmcite,2099,843,5434,25,HumanHTO2,batch1,Mono/DC,CD14 Mono,0.568508,Mono/DC,Mono/DC
...,...,...,...,...,...,...,...,...,...,...,...,...
b_TTTGTCATCCGAGCCA-1,bmcite,3000,895,5403,25,HumanHTO8,batch2,T cell,CD4 Memory,0.296002,CD4 T,CD4 T
b_TTTGTCATCCGTAGGC-1,bmcite,2230,649,4744,25,HumanHTO9,batch2,T cell,CD4 Naive,0.429053,CD4 T,CD4 T
b_TTTGTCATCCTCGCAT-1,bmcite,4259,1457,6600,25,HumanHTO6,batch2,Mono/DC,CD16 Mono,0.506296,Mono/DC,Mono/DC
b_TTTGTCATCGCCGTGA-1,bmcite,2648,733,4939,25,HumanHTO3,batch2,T cell,CD4 Naive,0.234427,CD4 T,CD4 T


# Dataset Class

In [13]:
class scRNACSV(Dataset):
  def __init__(self, expr, meta, transform = None, target_transform=None):
    self.expr = expr
    self.meta = meta
    self.cells = list(expr.columns)
    self.labels = list(meta["perturb"])
    self.samples = [(self.cells,self.labels[i]) for i in range(len(self.labels))]
    self.label_keys = list(set(self.labels))
    self.label_keys.sort()
    self.label_dic = {}
    for label, i in zip(self.label_keys, range(len(self.label_keys))):
      self.label_dic[label] = i
    print(f"This is the label dictionary of this dataset {self.label_dic}")
    self.transform = transform
    self.target_transform = target_transform
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    if self.transform:
      ret = self.transform(one_cell)
    else:
      ret = one_cell
    lab = self.label_dic[self.labels[idx]]
    
    return ret, self.label_dic[self.labels[idx]]

In [14]:
 class DINO_Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    
  def __call__(self, x):
    inputs = []
    data = x  
    length = x.shape[0]
    #print("This is the length")
    #print(length)
    global_size = int(length * self.global_crops_scale)
    global_index_1 = np.random.choice(length, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    
    global_data_1 = data[global_index_1,]
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    return global_input_1

## Initiate Crop

In [15]:
crop = DINO_Crop(0.5,0.25,8)

In [16]:
dataset = scRNACSV(expr, meta, transform = crop)

This is the label dictionary of this dataset {'B cell': 0, 'CD4 T': 1, 'CD8 T': 2, 'MAIT': 3, 'Mono/DC': 4, 'NK': 5, 'Progenitor cells': 6, 'gdT': 7}


## Split dataset

In [17]:
train_length = int(len(dataset) * 0.8)
test_length = len(dataset) - train_length
train_dataset, test_dataset = torch.utils.data.random_split(dataset=dataset, lengths = [train_length, test_length], generator=torch.Generator().manual_seed(42))

## Dataloader

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle = True)

# Supervised Train

In [19]:
def train_one_epoch(trainloader, simple_model, epoch, optimizer):
  running_loss = 0.0
  simple_model.train()
  lossF = nn.CrossEntropyLoss()
  sum_loss = 0
  num = 0
  for batch_idx, (data, label) in enumerate(trainloader):
    data, label = Variable(data).cuda(), Variable(label).cuda()
    #data, label = Variable(data), Variable(label)
    optimizer.zero_grad()
    output = simple_model(data)
    loss = lossF(output, label)
    sum_loss += loss
    num += 1
    loss.backward()
    optimizer.step()
    #plot_grad_flow(simple_model.named_parameters())
    if batch_idx % 100 == 0:
      print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss {:.6f}".format(
                epoch, batch_idx*len(data), len(trainloader.dataset),
                100.*batch_idx/len(trainloader), loss.item()))
  loss_stat = (sum_loss/num).detach().cpu().numpy().item()
  log_stats = {**{f'Average Loss': loss_stat}, 'epoch': epoch + 1}
  with (Path(output_dir) / "loss.txt").open("a") as f: f.write(json.dumps(log_stats) + "\n")

In [20]:
os.chdir("/content/drive/Shareddrives/Documentation/Jiaxin_Li/Supervised_perturb/")

## Train loop

In [21]:
os.chdir("/content/drive/Shareddrives/Documentation/Jiaxin_Li/Supervised_perturb/")
!rm -r ./128+128_bmcite_no_scale_Aug30

In [ ]:
os.chdir("/content/drive/Shareddrives/Documentation/Jiaxin_Li/Supervised_perturb/")
output_dir = './128+128_bmcite_no_scale_Aug30' ### customized your own output_dir
model = vits.vit_tiny()
optimizer = optim.Adam(model.parameters(), lr = 1e-2)
model.cuda()
Path(output_dir).mkdir(parents=True, exist_ok=True)
saveckp_freq = 1
to_restore = {"epoch": 0}
utils.restart_from_checkpoint(
    os.path.join(output_dir, "checkpoint.pth"),
    run_variables=to_restore,
    model=model,
    optimizer=optimizer,
)
start_epoch = to_restore["epoch"]
for epoch in range(start_epoch, 1000):
    model.train()
    train_one_epoch(train_loader, model, epoch, optimizer)
    # ============ writing logs ... ============
    save_dict = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch + 1,
    }
    # Train loop
    if epoch % 3 == 0:
      correct, total = 0, 0
    
      with torch.no_grad():
        # Iterate over the test data and generate predictions
          for i, data in enumerate(train_loader, 0):
          # Get inputs
            inputs, targets = data
            inputs = inputs.cuda()
            targets = targets.cuda()
          # Generate outputs
            model.eval()
            outputs = model(inputs)
          # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            #print(outputs)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            # Print accuracy
          train_acc = "%f %%" %(100 * correct / total)
          print(train_acc)
    # Testing loop
      correct, total = 0, 0
      with torch.no_grad():
        # Iterate over the test data and generate predictions
          for i, data in enumerate(test_loader, 0):
          # Get inputs
            inputs, targets = data
            inputs = inputs.cuda()
            #print(inputs.shape)
            #print(inputs)
            targets = targets.cuda()
          # Generate outputs
            model.eval()
            outputs = model(inputs)
          # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            #print(inputs[:,0:20])
            print(predicted)
            #print(outputs[:,0:10])
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        # Print accuracy
          test_acc =  "%f %%" %(100 * correct / total)
          print(test_acc)
      acc_stats = {"Training Accuracy": train_acc, "Testing Accuracy":  test_acc, "epoch": epoch+1}
      with (Path(output_dir) / "acc.txt").open("a") as f: f.write(json.dumps(acc_stats) + "\n")
    utils.save_on_master(save_dict, os.path.join(output_dir, 'checkpoint.pth'))
    if saveckp_freq and epoch % saveckp_freq == 0:
        utils.save_on_master(save_dict, os.path.join(output_dir, f'checkpoint{epoch:04}.pth'))

Train Epoch: 0 [0/24537 (0%)]	Loss 2.072412
Train Epoch: 0 [1600/24537 (7%)]	Loss 2.035679
Train Epoch: 0 [3200/24537 (13%)]	Loss 1.907763
Train Epoch: 0 [4800/24537 (20%)]	Loss 1.912887
Train Epoch: 0 [6400/24537 (26%)]	Loss 1.715126
